In [1]:
# !pip install firebase-admin

In [1]:
import os
import firebase_admin
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from firebase_admin import credentials, firestore
from datetime import datetime, timedelta
from dotenv import load_dotenv

# Firebase SDKの初期化（サービスアカウントJSONキーのパスを指定）
cred = credentials.Certificate("./smile-sync-firebase-service-account.json")
firebase_admin.initialize_app(cred)

# Firestoreクライアントを初期化
db = firestore.client()

### 1分ごとの笑顔時間の遷移

In [2]:
# コレクションとドキュメントへの参照
collection_name = "smilepoint_history"
document_name = "2024-10-31 03:15:32"
doc_ref = db.collection(collection_name).document(document_name)

# ドキュメントを取得
doc = doc_ref.get()
if doc.exists:
    smile_points_log = doc.to_dict().get('smile_points_log', [])
else:
    print(f"Document {document_name} not found in collection {collection_name}.")
    exit()

# データを確認
print(smile_points_log[:5])  # データが取得されているか最初の5つを確認

In [3]:
# データフレームに変換
df = pd.DataFrame(smile_points_log)

# 10分以内のデータに限定
df = df[df['since_meeting_start'] < 600]

# 1分ごとにデータを集計
df['minute_bucket'] = (df['since_meeting_start'] // 60).astype(int)


In [6]:
# .envから環境変数を読み込む
load_dotenv(override=True)
name1 = os.getenv("NICKNAME1")
name2 = os.getenv("NICKNAME2")
name3 = os.getenv("NICKNAME3")

# 特定の被験者のデータをフィルタリング
name1_df = df[df['nickname'] == name1]
name2_df = df[df['nickname'] == name2]
name3_df = df[df['nickname'] == name3]

# 1分ごとのカウントを計算
all_counts = df.groupby('minute_bucket')['smile_point'].sum()
name1_counts = name1_df.groupby('minute_bucket')['smile_point'].sum()
name2_counts = name2_df.groupby('minute_bucket')['smile_point'].sum()
name3_counts = name3_df.groupby('minute_bucket')['smile_point'].sum()

# グラフの作成準備（棒グラフの位置をずらすためにx軸の範囲を設定）
time_buckets = np.arange(10)
bar_width = 0.2

# グラフを描画
plt.figure(figsize=(12, 8))

# 全員のカウント（棒グラフ）
plt.bar(time_buckets, all_counts.values, width=bar_width, label="All Users", align='center', alpha=0.7)

# 各被験者のカウント（折れ線グラフ）
plt.plot(time_buckets, name1_counts.reindex(all_counts.index, fill_value=0).values, label="Name1", color='orange', marker='o', linewidth=2)
plt.plot(time_buckets, name2_counts.reindex(all_counts.index, fill_value=0).values, label="Name2", color='green', marker='o', linewidth=2)
plt.plot(time_buckets, name3_counts.reindex(all_counts.index, fill_value=0).values, label="Name3", color='red', marker='o', linewidth=2)

# グラフの装飾
plt.xlabel("Minutes from start")
plt.ylabel("Smile Point Count")
plt.title("Smile Points Count per Minute")
plt.xticks(time_buckets, labels=[f"{i} min" for i in time_buckets])

# 各バーの上に数値ラベルを追加
for i in range(len(all_counts)):
    plt.text(time_buckets[i], all_counts.values[i] + 0.1, str(all_counts.values[i]), ha='center', va='bottom', fontsize=9, color='blue')
    plt.text(time_buckets[i] + 0.2, name1_counts.reindex(all_counts.index, fill_value=0).values[i] + 0.1, str(name1_counts.reindex(all_counts.index, fill_value=0).values[i]), ha='center', va='bottom', fontsize=9, color='orange')
    plt.text(time_buckets[i] + 0.2, name2_counts.reindex(all_counts.index, fill_value=0).values[i] + 0.1, str(name2_counts.reindex(all_counts.index, fill_value=0).values[i]), ha='center', va='bottom', fontsize=9, color='green')
    plt.text(time_buckets[i] + 0.2, name3_counts.reindex(all_counts.index, fill_value=0).values[i] + 0.1, str(name3_counts.reindex(all_counts.index, fill_value=0).values[i]), ha='center', va='bottom', fontsize=9, color='red')

# 各被験者の合計値を計算して表示
total_all = all_counts.sum()
total_name1 = name1_counts.sum()
total_name2 = name2_counts.sum()
total_name3 = name3_counts.sum()

plt.text(len(all_counts) - 3, max(all_counts) - 80, f"Total (All): {total_all}", ha='center', va='bottom', fontsize=10, color='blue')
plt.text(len(all_counts) - 3, max(all_counts) + 10, f"Total (Name1): {total_name1}", ha='center', va='bottom', fontsize=10, color='orange')
plt.text(len(all_counts) - 3, max(all_counts) - 20, f"Total (Name2): {total_name2}", ha='center', va='bottom', fontsize=10, color='green')
plt.text(len(all_counts) - 3, max(all_counts) - 50, f"Total (Name3): {total_name3}", ha='center', va='bottom', fontsize=10, color='red')

# 凡例を表示
plt.legend()

# グラフを表示
save_dir = os.getenv("SAVE_DIR")
plt.tight_layout()
plt.savefig(save_dir + 'smile_points_per_minute.png', dpi=300)
plt.show()

# 各時間のdfを作成
output_df = pd.DataFrame({
    'minute_bucket': time_buckets,
    'all_users': all_counts.values,
    name1: name1_counts.reindex(all_counts.index, fill_value=0).values,
    name2: name2_counts.reindex(all_counts.index, fill_value=0).values,
    name3: name3_counts.reindex(all_counts.index, fill_value=0).values,
})

# 平均値を計算
output_df['average'] = output_df[['all_users', name1, name2, name3]].mean(axis=1)

# 10分間の合計値を追加
total_row = {
    'minute_bucket': 'Total',
    'all_users': all_counts.values.sum(),
    name1: name1_counts.sum(),
    name2: name2_counts.sum(),
    name3: name3_counts.sum(),
    'average': output_df['average'].mean()
}
output_df = pd.concat([output_df, pd.DataFrame([total_row])], ignore_index=True)

# CSV出力
output_csv_path = os.path.join(save_dir, 'smile_points_per_minute.csv')
output_df.to_csv(output_csv_path, index=False)
print(f"CSV file saved to {output_csv_path}")

### 1分ごとのアイデア数の遷移

In [ ]:
# ドキュメントを取得
doc = doc_ref.get()
if doc.exists:
    smile_ideas_log = doc.to_dict().get('smile_ideas_log', [])
else:
    print(f"Document {document_name} not found in collection {collection_name}.")
    exit()

# データを確認
print(smile_ideas_log[:5])  # データが取得されているか最初の5つを確認

In [ ]:
# データフレームに変換
df = pd.DataFrame(smile_ideas_log)

# 10分以内のデータに限定
df = df[df['since_meeting_start'] < 600]

# 1分ごとにデータを集計
df['minute_bucket'] = (df['since_meeting_start'] // 60).astype(int)


In [ ]:
# .envから環境変数を読み込む
load_dotenv(override=True)
name1 = os.getenv("NICKNAME1")
name2 = os.getenv("NICKNAME2")
name3 = os.getenv("NICKNAME3")

# 特定の被験者のデータをフィルタリング
name1_df = df[df['nickname'] == name1]
name2_df = df[df['nickname'] == name2]
name3_df = df[df['nickname'] == name3]

# 1分ごとのカウントを計算
all_counts = df.groupby('minute_bucket')['smile_point'].sum()
name1_counts = name1_df.groupby('minute_bucket')['smile_point'].sum()
name2_counts = name2_df.groupby('minute_bucket')['smile_point'].sum()
name3_counts = name3_df.groupby('minute_bucket')['smile_point'].sum()

# グラフの作成準備（棒グラフの位置をずらすためにx軸の範囲を設定）
time_buckets = np.arange(10)
bar_width = 0.2

# グラフを描画
plt.figure(figsize=(12, 8))

# 全員のカウント（棒グラフ）
plt.bar(time_buckets, all_counts.values, width=bar_width, label="All Users", align='center', alpha=0.7)

# 各被験者のカウント（折れ線グラフ）
plt.plot(time_buckets, name1_counts.reindex(all_counts.index, fill_value=0).values, label="Name1", color='orange', marker='o', linewidth=2)
plt.plot(time_buckets, name2_counts.reindex(all_counts.index, fill_value=0).values, label="Name2", color='green', marker='o', linewidth=2)
plt.plot(time_buckets, name3_counts.reindex(all_counts.index, fill_value=0).values, label="Name3", color='red', marker='o', linewidth=2)

# グラフの装飾
plt.xlabel("Minutes from start")
plt.ylabel("Smile Point Count")
plt.title("Smile Points Count per Minute")
plt.xticks(time_buckets, labels=[f"{i} min" for i in time_buckets])

# 各バーの上に数値ラベルを追加
for i in range(len(all_counts)):
    plt.text(time_buckets[i], all_counts.values[i] + 0.1, str(all_counts.values[i]), ha='center', va='bottom', fontsize=9, color='blue')
    plt.text(time_buckets[i] + 0.2, name1_counts.reindex(all_counts.index, fill_value=0).values[i] + 0.1, str(name1_counts.reindex(all_counts.index, fill_value=0).values[i]), ha='center', va='bottom', fontsize=9, color='orange')
    plt.text(time_buckets[i] + 0.2, name2_counts.reindex(all_counts.index, fill_value=0).values[i] + 0.1, str(name2_counts.reindex(all_counts.index, fill_value=0).values[i]), ha='center', va='bottom', fontsize=9, color='green')
    plt.text(time_buckets[i] + 0.2, name3_counts.reindex(all_counts.index, fill_value=0).values[i] + 0.1, str(name3_counts.reindex(all_counts.index, fill_value=0).values[i]), ha='center', va='bottom', fontsize=9, color='red')

# 各被験者の合計値を計算して表示
total_all = all_counts.sum()
total_name1 = name1_counts.sum()
total_name2 = name2_counts.sum()
total_name3 = name3_counts.sum()

plt.text(len(all_counts) - 3, max(all_counts) - 80, f"Total (All): {total_all}", ha='center', va='bottom', fontsize=10, color='blue')
plt.text(len(all_counts) - 3, max(all_counts) + 10, f"Total (Name1): {total_name1}", ha='center', va='bottom', fontsize=10, color='orange')
plt.text(len(all_counts) - 3, max(all_counts) - 20, f"Total (Name2): {total_name2}", ha='center', va='bottom', fontsize=10, color='green')
plt.text(len(all_counts) - 3, max(all_counts) - 50, f"Total (Name3): {total_name3}", ha='center', va='bottom', fontsize=10, color='red')

# 凡例を表示
plt.legend()

# グラフを表示
save_dir = os.getenv("SAVE_DIR")
plt.tight_layout()
plt.savefig(save_dir + 'smile_points_per_minute.png', dpi=300)
plt.show()

# 各時間のdfを作成
output_df = pd.DataFrame({
    'minute_bucket': time_buckets,
    'all_users': all_counts.values,
    name1: name1_counts.reindex(all_counts.index, fill_value=0).values,
    name2: name2_counts.reindex(all_counts.index, fill_value=0).values,
    name3: name3_counts.reindex(all_counts.index, fill_value=0).values,
})

# 平均値を計算
output_df['average'] = output_df[['all_users', name1, name2, name3]].mean(axis=1)

# 10分間の合計値を追加
total_row = {
    'minute_bucket': 'Total',
    'all_users': all_counts.values.sum(),
    name1: name1_counts.sum(),
    name2: name2_counts.sum(),
    name3: name3_counts.sum(),
    'average': output_df['average'].mean()
}
output_df = pd.concat([output_df, pd.DataFrame([total_row])], ignore_index=True)

# CSV出力
output_csv_path = os.path.join(save_dir, 'smile_points_per_minute.csv')
output_df.to_csv(output_csv_path, index=False)
print(f"CSV file saved to {output_csv_path}")